In [ ]:
import pymysql
import pandas as pd
from surprise import Dataset, Reader, SVD
from pymongo import MongoClient


# 连接MySQL数据库并读取数据
def fetch_data_from_mysql():
    connection = pymysql.connect(
        host='your_mysql_host',
        user='your_mysql_user',
        password='your_mysql_password',
        db='your_database_name'
    )
    query = "SELECT user_id, item_id, rating FROM ratings"
    df = pd.read_sql(query, connection)
    connection.close()
    return df


# 计算推荐结果
def calculate_recommendations(df):
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)

    trainset = data.build_full_trainset()
    algo = SVD()
    algo.fit(trainset)

    recommendations = {}
    user_ids = df['user_id'].unique()
    for user_id in user_ids:
        user_recommendations = []
        for item_id in df['item_id'].unique():
            if not trainset.knows_user(user_id) or not trainset.knows_item(item_id):
                continue
            pred = algo.predict(user_id, item_id)
            user_recommendations.append((item_id, pred.est))
        user_recommendations.sort(key=lambda x: x[1], reverse=True)
        recommendations[user_id] = [item_id for item_id, _ in user_recommendations[:10]]  # 推荐前10个
    return recommendations


# 将推荐结果写入MongoDB
def save_recommendations_to_mongodb(recommendations):
    client = MongoClient('mongodb://localhost:27017/')
    db = client['recommendation_db']
    collection = db['recommendations']
    for user_id, recs in recommendations.items():
        collection.update_one({'user_id': user_id}, {'$set': {'recommendations': recs}}, upsert=True)
    client.close()


# 主函数
def main():
    df = fetch_data_from_mysql()
    recommendations = calculate_recommendations(df)
    save_recommendations_to_mongodb(recommendations)


if __name__ == "__main__":
    main()
